In [97]:
from typing import Tuple, List
import apache_beam as beam
import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam import CoGroupByKey, PCollection
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner

In [98]:
p = beam.Pipeline(InteractiveRunner())

In [121]:
_input: PCollection[Tuple[str, int]] = p | "Create input" >> beam.Create([('a', 1), ('b', 2), ('b', 3), ('c', 4)])
ib.collect(_input)

,0,1
0,a,1
1,b,2
2,b,3
3,c,4


In [122]:
multipliers = p | "Create multipliers" >> beam.Create([('b', 10), ('c', 100)])
ib.collect(multipliers)

,0,1
0,b,10
1,c,100


In [123]:
grouped_by_key: PCollection[Tuple[str, Tuple[List[int], List[int]]]] = (_input, multipliers) | CoGroupByKey()
ib.collect(grouped_by_key)

,0,1
0,b,"([2, 3], [10])"
1,c,"([4], [100])"
2,a,"([1], [])"


In [124]:
class SumAndMultiply(beam.DoFn):
    DEFAULT_MULTIPLIER = 1

    def process(self, element: Tuple[str, Tuple[List[int], List[int]]], *args, **kwargs):
        key: str = element[0]
        val: Tuple[List[int], List[int]] = element[1]
        _left: List[int] = val[0]
        _right: List[int] = val[1]
        _multiplier = _right[0] if _right and len(_right) > 0 else self.DEFAULT_MULTIPLIER
        yield key, sum(_left) * _multiplier

In [125]:
grouped_and_sum_multiply: PCollection[Tuple[str, int]] = grouped_by_key | beam.ParDo(SumAndMultiply())
ib.collect(grouped_and_sum_multiply)

,0,1
0,b,50
1,c,400
2,a,1
